In [1]:
# Loading Libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import random
random.seed(42)

import syft as sy

In [2]:
# Creating Hook
hook = sy.TorchHook(torch)

In [3]:
# Creating 10 virtual workers
no_workers = 10

vw = []
for i in range(no_workers):
    vw.append(sy.VirtualWorker(id="w%d"%(i+1), hook=hook))

In [4]:
# Loading MNIST data
fed_train_loader = sy.FederatedDataLoader(datasets.MNIST('../data', train=True, download=True,
                                               transform=transforms.Compose([
                                                   transforms.ToTensor(), 
                                                   transforms.Normalize((0.1307,), (0.3081,))])).federate(tuple(vw)), 
                                                batch_size=64,
                                                shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=False, 
                                                         transform=transforms.Compose([
                                                             transforms.ToTensor(), 
                                                             transforms.Normalize((0.1307,), (0.3081,))])),
                                          batch_size=1000, 
                                          shuffle=True)

In [5]:
class Net(nn.Module):
    
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [40]:
available_clients = 4
a = list(range(0, len(vw)))
random.shuffle(a)
selected_clients = a[:available_clients]

[7, 3, 2, 8]

In [49]:
def train(model, device, federated_train_loader, optimizer, epoch):
    
    model.train()
    
    c = 0
    
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        
        if data.location.id[-1] not in selected_clients:
            continue
        
        model.send(data.location)
        data, target = data.to(device), target.to(device)
        c+=len(data)
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = F.nll_loss(output, target)
        
        loss.backward()
        
        optimizer.step()
        
        model.get()
    
        if batch_idx % 100 == 0:
            loss = loss.get()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))
            
    print("Trained on %d." %(c))

In [ ]:
# Define 